# Import Libraries

In [1]:
import pandas as pd
import geopandas as gpd
import os

In [2]:
os.chdir('/Users/sabbirdelowar/projects/climate-hazard')

# Define Paths

In [3]:
district_shp = "data/aoi/aoi-multipoly.shp"

flood_csv = "results/individual-hazards/combined-flood-hazard.csv"
heat_csv = "results/individual-hazards/heat-hazard.csv"

output_dir = "results/total-hazard/"
os.makedirs(output_dir, exist_ok=True)

output_csv = os.path.join(output_dir, "total-hazard.csv")
output_shp = os.path.join(output_dir, "total-hazard.shp")

# Load and merge files

In [18]:
gdf = gpd.read_file(district_shp)
gdf = gdf[['name', 'geometry']].copy()
gdf = gdf.rename(columns={'name': 'district'})

flood_df = pd.read_csv(flood_csv)
heat_df = pd.read_csv(heat_csv)

hazard_df = gdf.merge(flood_df, on='district', how='left')
hazard_df = hazard_df.merge(heat_df, on='district', how='left')
print(hazard_df.head())

    district                                           geometry  \
0   Bagerhat  MULTIPOLYGON (((89.84021 21.82784, 89.84052 21...   
1  Bandarban  POLYGON ((92.38065 22.33064, 92.38159 22.33002...   
2    Barguna  MULTIPOLYGON (((89.87968 22.07869, 89.87865 22...   
3    Barisal  MULTIPOLYGON (((90.55429 22.59271, 90.55435 22...   
4      Bhola  MULTIPOLYGON (((90.77172 21.85595, 90.76920 21...   

   heavy_rain_months  rainfall_hazard  hand_hazard  combined_flood_hazard  \
0                  9          0.28125       0.5450               0.413125   
1                 20          0.62500       0.5398               0.582400   
2                  7          0.21875       0.3734               0.296075   
3                  1          0.03125       0.4516               0.241425   
4                 12          0.37500       0.3461               0.360550   

   heatwave_days  heat_hazard  
0            111     0.516779  
1            183     1.000000  
2             52     0.120805  
3     

# Total hazard score

In [19]:
hazard_df['total_hazard'] = (
    hazard_df['combined_flood_hazard'] + 
    hazard_df['heat_hazard']
)

hazard_df = hazard_df.rename(columns={
    'rainfall_hazard': 'rain_h',
    'hand_hazard': 'hand_h',
    'combined_flood_hazard': 'flood_h',
    'heat_hazard': 'heat_h',
    'total_hazard': 'total_h'
})

# Optionally normalize 0–1
max_total = hazard_df['total_h'].max()
hazard_df['total_h'] = hazard_df['total_h'] / max_total
cols = ['district', 'rain_h', 'hand_h', 'flood_h', 'heat_h', 'total_h', 'geometry']
hazard_df = hazard_df[cols]

hazard_df_nogeo = hazard_df.drop(columns='geometry')
hazard_df_nogeo.to_csv(output_csv, index=False)
hazard_df.to_file(output_shp)